# EDA jobsearch about Säkerhet och bevakning

In [ ]:
import requests 
import json 


url = "https://jobsearch.api.jobtechdev.se"
url_for_search = f"{url}/search"


def _get_ads(params):
    headers = {'accept': 'application/json'}
    response = requests.get(url_for_search, headers=headers, params=params)
    response.raise_for_status()  # check for http errors
    return json.loads(response.content.decode('utf8'))


def example_search_return_number_of_hits(query):
    # limit: 0 means no ads, just a value of how many ads were found.
    search_params = {'q': query, 'limit': 0}
    json_response = _get_ads(search_params)
    number_of_hits = json_response['total']['value']
    print(f"\nNumber of hits = {number_of_hits}")


def example_search_loop_through_hits(query):
    # limit = 100 is the max number of hits that can be returned.
    # If there are more (which you find with ['total']['value'] in the json response)
    # you have to use offset and multiple requests to get all ads.
    search_params = {'q': query, 'limit': 100}
    json_response = _get_ads(search_params)
    hits = json_response['hits']
    for hit in hits:
        print(f"{hit['headline']}, {hit['employer']['name']}")


if __name__ == '__main__':
    query = 'säkerhet och bevakning'
    example_search_loop_through_hits(query)
    example_search_return_number_of_hits(query)
    
    

In [ ]:
search_params = {'q': query, 'limit': 100}
json_response = _get_ads(search_params)

json_response.keys()

In [ ]:
type(json_response)

In [ ]:
json_response["hits"]

In [ ]:
json_response["hits"][5]["headline"], json_response["hits"][5]["number_of_vacancies"], json_response["hits"][5]["salary_type"]["label"]

In [ ]:
json_response["hits"][0]["employer"]

In [ ]:
json_response["hits"][0].keys()

In [ ]:
json_response["hits"][-1]["employer"]

In [ ]:
json_response["hits"]

name = []
for i in json_response["hits"]:
    name.append(i["employment_type"]["label"])


print(name)

    

In [ ]:
from pandas import json_normalize
import pandas as pd
data = json_response["hits"]
df = pd.json_normalize(data)
df.head()

In [ ]:
new_df = df[["id", "employer.name", "employer.workplace", "workplace_address.country", "number_of_vacancies", "timestamp", "occupation.label", "workplace_address.region"]]

new_df.head()

In [ ]:
new_df.info()

In [ ]:

# using groupby and sort to get out number available vacacancies
top_10_vacancies = new_df.groupby("employer.name")["number_of_vacancies"].sum().sort_values(ascending= False).reset_index(name= "number_vacancies")

top_10_vacancies = top_10_vacancies.head(10)
top_10_vacancies

# Geojson data: get name and länskod

In [ ]:
from pathlib import Path
import json

working_directory = Path.cwd().parent.parent

path_geojson = working_directory / "geojson_data"



with open(path_geojson / "swedish_regions.geojson", "r", encoding= "utf-8") as file:
    json_data = json.load(file)
    
    
# json_data

In [ ]:
def region_code(geojson):

    properties = [property.get("properties") for property in geojson.get("features")]

    region_länskod = {name_code.get("name"): name_code.get("ref:se:länskod")for name_code in properties}
    
    return region_länskod

In [ ]:
print(region_code(geojson= json_data))

# Bring data from database from snowflake

In [ ]:
from dotenv import load_dotenv
import os 
import pandas as pd 
from snowflake.connector import connect

load_dotenv()

def data_snowflake(occupation_field):
    with connect(
        user = os.getenv("SNOWFLAKE_USER"),
        password = os.getenv("SNOWFLAKE_PASSWORD"),
        account = os.getenv("SNOWFLAKE_ACCOUNT"),
        warehous = os.getenv("SNOWFLAKE_WAREHOUSE"),
        database = os.getenv("SNOWFLAKE_DATABASE"),
        schema = os.getenv("SNOWFLAKE_SCHEMA"),
        role = os.getenv("SNOWFLAKE_ROLE")
    ) as conn:
        
        if occupation_field == "data/it":
            query = 'SELECT * FROM mart_data_it'
        elif occupation_field == "säkerhet och bevakning":
            query = 'SELECT * FROM mart_safety'
        elif occupation_field == "yrken med social inriktning":
            query = 'SELECT * FROM mart_social_work'
        
        df = pd.read_sql(query, conn)
    
        return df 

In [ ]:
data_it = data_snowflake("data/it")

In [ ]:
data_it.groupby("WORKPLACE_REGION")["NUMBER_OF_VACANCIES"].sum()

In [ ]:
data_it[data_it["WORKPLACE_REGION"] == "ej specificerad"]["NUMBER_OF_VACANCIES"].sum()

In [ ]:
data_it.query("WORKPLACE_REGION == 'ej specificerad'")["NUMBER_OF_VACANCIES"].sum()

In [ ]:


def region_vacancies(occupation, json_data = json_data):
    
    df_occupation = data_snowflake(occupation_field= occupation)
    
    
    df_region_vacancies = df_occupation[["WORKPLACE_REGION", "NUMBER_OF_VACANCIES"]]
    
    df = df_region_vacancies.groupby("WORKPLACE_REGION")["NUMBER_OF_VACANCIES"].sum().reset_index(name="VACANCIES").sort_values(by= "VACANCIES").reset_index(drop=True)
    
    df = df.drop(index=["ej specificerad"], errors= "ignore").reset_index(drop= True)
    
    # if one or more region dosen't have any data, this is a safty to make sure i set them to zero
    missing_data = pd.DataFrame({
        "WORKPLACE_REGION": region_code(json_data).keys()
    })
    
    df_merge = missing_data.merge(df, on= "WORKPLACE_REGION", how= "left")
    
    df_merge["VACANCIES"] = df_merge["VACANCIES"].fillna(0).astype(int)
    
    
    
    
    return df_merge

In [ ]:
df_data_it = region_vacancies(occupation= "data/it") 

type(df_data_it)

In [ ]:
df_data_it

# Use difflib and function get_close_matches

In [ ]:
from difflib import get_close_matches

def get_matches(occupation, json_data= json_data):
    
    df_occupation = region_code(geojson= json_data)
    
    # get all region in a list
    regions = region_vacancies(occupation= occupation, json_data=json_data)["WORKPLACE_REGION"].values
    
    
    region_code_map = []
    for region in regions:

        name_macthes = get_close_matches(region, df_occupation.keys(), n= 1)[0]
        code = df_occupation[name_macthes]
        region_code_map.append(code)
        
            
        
        
    return region_code_map

In [ ]:
print(region_code(geojson= json_data))

print(get_matches(occupation= "data/it"))

print(df_data_it["WORKPLACE_REGION"].values)

# create choropleth map

In [ ]:
# make value  colorscale logarithmic
import numpy as np
def log_value(df, value):
    

    
    df["LOG_VALUE"] = np.log(df[value] + 1)
    
    return df["LOG_VALUE"] 

In [ ]:
from plotly import graph_objects as go 




def choroplth_map(occupation, data):
    fig = go.Figure(
        go.Choroplethmapbox(
            geojson= json_data,
            locations= get_matches(occupation= occupation),
            z= log_value(df= region_vacancies(occupation= occupation, json_data= json_data), value= "VACANCIES"),
            featureidkey= "properties.ref:se:länskod",
            colorscale="blues",
            marker_opacity = 0.9,
            marker_line_width = 0.4,
            marker_line_color = 'darkgrey',
            text= region_vacancies(occupation= occupation)["WORKPLACE_REGION"],
            customdata= region_vacancies(occupation=occupation)["VACANCIES"],
            hovertemplate= "<b>%{text}</b><br>Total vacancies: %{customdata}<extra></extra>",
            showscale= False
        )
    )
    
    fig.update_layout(
        title = dict(
            text=f"""
                <b>Antal lediga tjänster
                <br>för yrket {occupation}</b> 
                <br>Totala lediga tjänster: {data['NUMBER_OF_VACANCIES'].sum()}
                <br>Lediga tjänster som är 
                <br>ej specificerad regioner: {data.query("WORKPLACE_REGION == 'ej specificerad'")["NUMBER_OF_VACANCIES"].sum()}""",
                font= dict(size= 14)
        ),
        mapbox= dict(center= dict(lat= 62.6952, lon= 13.9149), style= "white-bg", zoom= 3.5),
        margin= dict(r=0, t= 50, l= 0, b= 10),
        dragmode= False,
        width= 500,
        height= 530
    )
    
    return fig

In [ ]:
fig = choroplth_map(occupation= "data/it", data = data_it)

fig.show()